In [1]:
from validate_adrresses import validate_address
import pandas as pd
import os
from tqdm import tqdm
print(os.getcwd())

/home/luis/Documents/Belcorp/iteration_adresses


In [2]:
df = pd.read_csv('./data/direcciones.csv', sep=';')

/tmp/ipykernel_3734/4095483860.py:1: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/direcciones.csv', sep=';')


In [3]:
cols = [col for col in df]
col1 = cols[:3]
col2 = cols[3:]

cols = []
for col in col1:
    cols.append(col)
cols.append("NUEVA DIRECCION")
for col in col2:
    cols.append(col)

In [4]:
df["NUEVA DIRECCION"] = None
df = df.reindex(columns=cols)
df = df[df["IND TERRITORIOS"] == 9]

In [5]:
df_tmp = df[["DIRECCION DE DOMICILIO", "COMUNA DOMICILIO", "NUEVA DIRECCION", "X", "Y"]]
df_tmp = df_tmp[6000:]
df_tmp

,DIRECCION DE DOMICILIO,COMUNA DOMICILIO,NUEVA DIRECCION,X,Y
28959,AV. MONICA BORDEU VIRQUENCO SITIO 46,LOS ANGELES,None,,
28960,LAURITA ACUÑA CASA 1401,NACIMIENTO,None,,
28963,CODPA2191 SECTOR TUCAPEL,ARICA,None,,
28976,CHEPICA POBLACION VILLA JARDIN PAREDONES DE AU...,CHIMBARONGO,None,,
28979,CAMP. LA QUEBRADA DE LA CHIMBA CASA 36 AGRUPAC...,ANTOFAGASTA,None,,
...,...,...,...,...,...
33051,ARTURO PRAT 510 CASA DOS PISOS,SAN ANTONIO,None,-38.7471312,-72.9524371
33067,ISABEL RODAS POBLACION,VALDIVIA,None,,
33069,LORD COCHRANE 611 CERRO ARENAS,SAN ANTONIO,None,,
33070,EL ARADO 504 VILLA OBLONERA,LA FLORIDA,None,,


In [6]:
arr_df = []
itr = 5
step = len(df_tmp)//itr
for i in range(itr):
    if i < itr - 1:
        arr_df.append(df_tmp[step*i:step*(i+1)])
    else:
        arr_df.append(df_tmp[step*i:])
arr_df[-1]

,DIRECCION DE DOMICILIO,COMUNA DOMICILIO,NUEVA DIRECCION,X,Y
32297,PUDAHUEL 9018,PUDAHUEL,None,,
32299,"VILLAGE CASA DE DOS PISOS ROJA, DOS ALAMOS ESQ...",TEMUCO,None,,
32308,PJE. 4 MALLOQUITO,PEÑAFLOR,None,,
32311,PASAJE GIRASOL SAN EXPEDITO PASJE GIRASOL CAS...,LA LIGUA,None,,
32313,LA RUDA PTE. CALLE MANZAN H CASA 23 CERRO MARI...,VALPARAISO,None,,
...,...,...,...,...,...
33051,ARTURO PRAT 510 CASA DOS PISOS,SAN ANTONIO,None,-38.7471312,-72.9524371
33067,ISABEL RODAS POBLACION,VALDIVIA,None,,
33069,LORD COCHRANE 611 CERRO ARENAS,SAN ANTONIO,None,,
33070,EL ARADO 504 VILLA OBLONERA,LA FLORIDA,None,,


In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os 

from multiprocessing import Pool
def parallel(df_):
    num_cores = os.cpu_count()  # Dynamically get the number of CPU cores
    print(num_cores)
    ddata = dd.from_pandas(df_, npartitions=num_cores)  # Set partitions based on available cores

    meta = pd.DataFrame({
        "DIRECCION DE DOMICILIO": pd.Series(dtype='str'),
        "COMUNA DOMICILIO": pd.Series(dtype='str'),
        "NUEVA DIRECCION": pd.Series(dtype='str'),
        "X": pd.Series(dtype='float'),
        "Y": pd.Series(dtype='float'),
        "DIRECCION TOTAL": pd.Series(dtype='str')
    })

    def apply_validate_address(df_chunk):
        df_chunk[["DIRECCION DE DOMICILIO", "COMUNA DOMICILIO"]] = df_chunk[["DIRECCION DE DOMICILIO", "COMUNA DOMICILIO"]].fillna("")
        
        # Apply the validation function in parallel
        df_chunk[["NUEVA DIRECCION", "X", "Y"]] = df_chunk.apply(
            lambda row: pd.Series(validate_address(row["DIRECCION DE DOMICILIO"] + " " + row["COMUNA DOMICILIO"])),
            axis=1
        )
        return df_chunk

    res = ddata.map_partitions(apply_validate_address, meta=meta).compute()
    return res

def parallel_wrapper(df):
    return parallel(df)

if __name__ == '__main__':
    arr_df_tmp = []
    with Pool(processes=os.cpu_count()) as pool:  # Dynamically set pool size based on CPU cores
        arr_df_tmp = pool.map(parallel_wrapper, arr_df)

2424242424




['ALCALDE', 'GABRIEL', 'SANCHEZ', 'SILVA', '1693,', 'COLLIPULLI,', 'ARAUCANIA', 'COLLIPULLI', 'COLLIPULLI']  ------  ALCALDE GABRIEL SANCHEZ SILVA 1693, COLLIPULLI, ARAUCANIA COLLIPULLI COLLIPULLI  -------  Alcalde Gabriel Sánchez, Collipulli, Provincia de Malleco, Región de la Araucanía, Chile
['PJ', 'ACONCAGUA', '0427', 'NUEVO', 'AMANECER', 'PUENTE', 'ALTO']  ------  PJ ACONCAGUA 0427 NUEVO AMANECER PUENTE ALTO  -------  Aconcagua, Población Nuevo Amanecer, Puente Alto, Provincia de Cordillera, Región Metropolitana de Santiago, 8150000, Chile
['AV.', 'NICANOR', 'GARCIA', 'PASAJE', 'BOMBERO', 'CASA', '19', 'PUERTO', 'VARAS']  ------  AV. NICANOR GARCIA PASAJE BOMBERO CASA 19 PUERTO VARAS  -------  Avenida Nicanor García, Población Pedro Aguirre Cerda II, Puerto Varas, Provincia de Llanquihue, Región de Los Lagos, 5550988, Chile
['LOS', 'JAZMINES', 'PLAZA', 'LOS', 'JASMINES', 'ALERCE', 'NORTE', 'PUERTO', 'MONTT']  ------  LOS JAZMINES PLAZA LOS JASMINES ALERCE NORTE PUER

In [ ]:
res

In [ ]:
print("A")

In [ ]:
res.to_csv('./data/out/ad7.csv')